In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
#Import basic libraries
%matplotlib inline


import os
import sys
import settings
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from django_pandas.io import read_frame
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.dates as mpl_dates

from matplotlib.dates import date2num

In [3]:
#Prepare to load stock data as pandas dataframe from source. In this case, prepare django
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from stocks.models import Listing, Stock

In [ ]:
#Get all the stocks listing
#Create a composite dataframe of closing prices for all securities 
#and compute correlation across all of them (and cluster them together)

def get_stock_listing(stock):
    import datetime
    first_date = datetime.date(2020, 1, 1)
    last_date = datetime.date(2020, 12, 31)
    listing = Listing.objects.filter(stock=stock, date__range=(first_date, last_date))
    df = read_frame(listing, fieldnames=['closing', 'date'], index_col='date')
    for column in df.columns:
        if column != 'stock':
           df[column] = pd.to_numeric(df[column]) 
    df = df.sort_index()
    df = df.reindex(columns = ['closing'])
    df.rename(columns={"closing":stock.sid.replace(' ', '_')}, inplace=True)
    #Optionally, filter out by date range
    #start_date = '2020-01-01'
    #end_date = '2021-12-31'
    #df = df.loc[start_date:end_date]
    return df

stocks = Stock.objects.all()
df = pd.DataFrame()
for stock in stocks:
    if stock.name != 'NIFTY' and stock.name != 'BANKNIFTY':
        sdf = get_stock_listing(stock)
        if len(sdf>0):
            #df = df.append(sdf)
            df = pd.concat([df, sdf])

for col in df.columns:
    print(col)

store = pd.HDFStore('store.h5')

store['df'] = df  # save it
#store['df']  # load it
print('Done')